In [77]:
import sys
import os
import shutil
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut
from src.databaseService.Merger import Merger_AV

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

In [78]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [79]:
ticker = "AAPL"
ts = TimeSeries(key=apiKey, output_format='pandas')
fullSharePrice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')

## Testing merge_shareprice

In [80]:
dates_init = ['2021-01-02', '2021-01-03']
_df_init = pd.DataFrame({
    'Date': dates_init,
    'Open': [100, 101],
    'High': [110, 111],
    'Low': [90, 91],
    'Close': [105, 106],
    'AdjClose': [105, 106],
    'Volume': [1000, 1100],
    'Dividends': [0, 0],
    'Splits': [1, 1]
})

def make_full_shareprice(dates, overrides=None):
    defaults = {
        '1. open': 10.0,
        '2. high': 11.0,
        '3. low': 9.0,
        '4. close': 10.5,
        '5. adjusted close': 10.5,
        '6. volume': 1000,
        '7. dividend amount': 0.0,
        '8. split coefficient': 1.0,
    }
    data = {}
    for key, val in defaults.items():
        if overrides and key in overrides:
            data[key] = overrides[key]
        else:
            data[key] = [val] * len(dates)
    idx = pd.DatetimeIndex(dates, name='date')
    return pd.DataFrame(data, index=idx)

In [81]:
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
merger.merge_shareprice(fullSharePrice)
result = asset.shareprice
assert len(result) > 2, f"Expected mroe than 2 rows, got {len(result)}"
resDateList = list(result['Date'].apply(lambda ts: dt.strptime(ts, '%Y-%m-%d').date()))
fullShareList = list(fullSharePrice.reset_index()['date'].apply(lambda ts: ts.date()))
assert all([a in resDateList for a in fullShareList]), "Dates do not match."
print('✅ Test 0.1 passed: New rows appended correctly.')

merger.merge_shareprice(fullSharePrice)
assert asset.shareprice.equals(result), f"Expected no changes, but asset state mutated: {asset.shareprice}" 
print('✅ Test 0.2 passed: Asset unchanged when no new rows.')


# Test 1: No new rows
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
asset_copy = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
full = make_full_shareprice([date(2021, 1, 2), date(2021, 1, 3)])
merger.merge_shareprice(full)
# asset.shareprice should be unchanged, so asset equals asset_copy
assert asset.shareprice.equals(asset_copy.shareprice), f"Expected no changes, but asset state mutated: {asset.shareprice}" 
print('✅ Test 1 passed: Asset unchanged when no new rows.')

# Test 2: Out-of-order input
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
dates = [date(2021, 1, 5), date(2021, 1, 4)]
full = make_full_shareprice(dates)
merger.merge_shareprice(full)
res_dates = [dt.strptime(d, '%Y-%m-%d').date() for d in asset.shareprice['Date']]
assert all(d in res_dates for d in dates), "Not all new dates present after merge."
assert res_dates == sorted(res_dates), "Dates are not sorted after merge."
print('✅ Test 2 passed: Out-of-order input merged and sorted correctly.')

# Test 3: Splits and dividends changes
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
asset_copy = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
chg_date = date(2021, 1, 2)
overrides = {
    '7. dividend amount': [0.5],
    '8. split coefficient': [2.0]
}
full = make_full_shareprice([chg_date], overrides)
merger.merge_shareprice(full)
assert asset.shareprice.equals(asset_copy.shareprice), f"Expected no changes, but asset state mutated: {asset.shareprice}" 
print('✅ Test 3 passed: Splits and dividends changes logged without new rows.')

# Test 4: String-vs-date resilience
# Introduce malformed date in existing data
_bad = _df_init.copy()
_bad.loc[0, 'Date'] = '2021/01/02'
asset = AssetData(ticker='TEST', shareprice=_bad)
merger = Merger_AV(asset)
try:
    full = make_full_shareprice([date(2021, 1, 2)])
    merger.merge_shareprice(full)
    raise AssertionError('Expected ValueError for malformed date, but none was raised.')
except ValueError:
    print('✅ Test 4 passed: Malformed date string raises ValueError.')

✅ Test 0.1 passed: New rows appended correctly.
✅ Test 0.2 passed: Asset unchanged when no new rows.
✅ Test 1 passed: Asset unchanged when no new rows.
✅ Test 2 passed: Out-of-order input merged and sorted correctly.
✅ Test 3 passed: Splits and dividends changes logged without new rows.
✅ Test 4 passed: Malformed date string raises ValueError.
